<h2> Hi guys.. I just to come back to share about "sampling" </h2>

normally we use 50k train data, it is seperate into 4 groups.
  > 1. True Positive (Target 1, Prediction 1) <br>
  > 2. True Negative (Target 0, Prediction 0) <br>
3. False Positive (Target 0, Prediction 1) <br>
4. False Negative (Target 1, Prediction 0) <br>


<h2> Important thing is we should avoid False Postive (FP), False Negative (FN). </h2>
<h2> if not, that will ruined our beautiful Classifier :(</h2>


<h3> So i made the pipeline for extract those sample datas. </h3>
<h3> and if you want you can also get data directly </h3>

please refered below, if it helpled please upvote :)
 > https://www.kaggle.com/leemop/amb04 <br> 
 > https://www.kaggle.com/leemop/amb59 <br>
 > https://www.kaggle.com/leemop/amb1014 <br>
 > https://www.kaggle.com/leemop/amb1519 <br>

1. Prepare pretrained model
 > We can train model in our own ways. <br>
 > or, you can add pretrained model in dataset: https://www.kaggle.com/leemop/b3-10epoch-fold0-alldata (if it helped please upvote :) )

2. at this kernel, using Pretrained Model, we filtered ultimate both FP, FN.
3. and saved to sampling_train*.tfrecords.
4. By doing that, You guys can add sampled data in your own Kernel.
5. And Please share great techniques for others :)

<h1> If it helped or informatives, please upvote for me. Thanks </h1>

In [ ]:
!pip install efficientnet tensorflow_addons > /dev/null
import os
import math
import random
import re
import warnings
from pathlib import Path
from typing import Optional, Tuple

import efficientnet.tfkeras as efn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_addons as tfa
from scipy.signal import get_window
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

NUM_FOLDS = 5
IMAGE_SIZE = 256
BATCH_SIZE = 32
EFFICIENTNET_SIZE = 3
WEIGHTS = "imagenet"

from kaggle_datasets import KaggleDatasets
from typing import Optional, Tuple

MIXUP_PROB = 0.0
EPOCHS = 5
R_ANGLE = 0 / 180 * np.pi
S_SHIFT = 0.0
T_SHIFT = 0.0
LABEL_POSITIVE_SHIFT = 0.99

def get_datapath():
    gcs_paths = []    
    for i, j in [(0, 4), (5, 9), (10, 14), (15, 19)]:        
        GCS_path = KaggleDatasets().get_gcs_path(f"g2net-waveform-tfrecords-train-{i}-{j}")        
        gcs_paths.append(GCS_path)
        print(GCS_path)

    all_files = []
    for path in gcs_paths:
        all_files.extend(np.sort(np.array(tf.io.gfile.glob(path + "/train*.tfrecords"))))        
           
    print("train_files: ", len(all_files))    
    return all_files

### Dataset Preperation

def create_cqt_kernels(
    q: float,
    fs: float,
    fmin: float,
    n_bins: int = 84,
    bins_per_octave: int = 12,
    norm: float = 1,
    window: str = "hann",
    fmax: Optional[float] = None,
    topbin_check: bool = True
) -> Tuple[np.ndarray, int, np.ndarray, float]:
    fft_len = 2 ** _nextpow2(np.ceil(q * fs / fmin))
    
    if (fmax is not None) and (n_bins is None):
        n_bins = np.ceil(bins_per_octave * np.log2(fmax / fmin))
        freqs = fmin * 2.0 ** (np.r_[0:n_bins] / np.float(bins_per_octave))
    elif (fmax is None) and (n_bins is not None):
        freqs = fmin * 2.0 ** (np.r_[0:n_bins] / np.float(bins_per_octave))
    else:
        warnings.warn("If nmax is given, n_bins will be ignored", SyntaxWarning)
        n_bins = np.ceil(bins_per_octave * np.log2(fmax / fmin))
        freqs = fmin * 2.0 ** (np.r_[0:n_bins] / np.float(bins_per_octave))
        
    if np.max(freqs) > fs / 2 and topbin_check:
        raise ValueError(f"The top bin {np.max(freqs)} Hz has exceeded the Nyquist frequency, \
                           please reduce the `n_bins`")
    
    kernel = np.zeros((int(n_bins), int(fft_len)), dtype=np.complex64)
    
    length = np.ceil(q * fs / freqs)
    for k in range(0, int(n_bins)):
        freq = freqs[k]
        l = np.ceil(q * fs / freq)
        
        if l % 2 == 1:
            start = int(np.ceil(fft_len / 2.0 - l / 2.0)) - 1
        else:
            start = int(np.ceil(fft_len / 2.0 - l / 2.0))

        sig = get_window(window, int(l), fftbins=True) * np.exp(
            np.r_[-l // 2:l // 2] * 1j * 2 * np.pi * freq / fs) / l
        
        if norm:
            kernel[k, start:start + int(l)] = sig / np.linalg.norm(sig, norm)
        else:
            kernel[k, start:start + int(l)] = sig
    return kernel, fft_len, length, freqs


def _nextpow2(a: float) -> int:
    return int(np.ceil(np.log2(a)))


def prepare_cqt_kernel(
    sr=22050,
    hop_length=512,
    fmin=32.70,
    fmax=None,
    n_bins=84,
    bins_per_octave=12,
    norm=1,
    filter_scale=1,
    window="hann"
):
    q = float(filter_scale) / (2 ** (1 / bins_per_octave) - 1)
    print(q)
    return create_cqt_kernels(q, sr, fmin, n_bins, bins_per_octave, norm, window, fmax) 


HOP_LENGTH = 16
cqt_kernels, KERNEL_WIDTH, lengths, _ = prepare_cqt_kernel(
                                        sr=2048,
                                        hop_length=HOP_LENGTH,
                                        fmin=20,
                                        fmax=1024,
                                        bins_per_octave=24)
LENGTHS = tf.constant(lengths, dtype=tf.float32)
CQT_KERNELS_REAL = tf.constant(np.swapaxes(cqt_kernels.real[:, np.newaxis, :], 0, 2))
CQT_KERNELS_IMAG = tf.constant(np.swapaxes(cqt_kernels.imag[:, np.newaxis, :], 0, 2))
PADDING = tf.constant([[0, 0],
            [KERNEL_WIDTH // 2, KERNEL_WIDTH // 2],
            [0, 0]])
            
def create_cqt_image(wave, hop_length=16, cqtCFG = None):
    CQTs = []
    for i in range(3):
        x = wave[i]
        x = tf.expand_dims(tf.expand_dims(x, 0), 2)
        x = tf.pad(x, PADDING, "REFLECT")

        CQT_real = tf.nn.conv1d(x, CQT_KERNELS_REAL, stride=hop_length, padding="VALID")
        CQT_imag = -tf.nn.conv1d(x, CQT_KERNELS_IMAG, stride=hop_length, padding="VALID")
        CQT_real *= tf.math.sqrt(LENGTHS)
        CQT_imag *= tf.math.sqrt(LENGTHS)

        CQT = tf.math.sqrt(tf.pow(CQT_real, 2) + tf.pow(CQT_imag, 2))
        CQTs.append(CQT[0])
    return tf.stack(CQTs, axis=2)      

def read_id_label_tfrecord(example):
    tfrec_format = {
        "wave": tf.io.FixedLenFeature([], tf.string),
        "wave_id": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, tfrec_format)
    return prepare_image(example["wave"], IMAGE_SIZE), example["wave_id"], tf.reshape(tf.cast(example["target"], tf.float32), [1])


def read_labeled_tfrecord(example):    
    tfrec_format = {
        "wave": tf.io.FixedLenFeature([], tf.string),
        "wave_id": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, tfrec_format)    
    return prepare_image(example["wave"], IMAGE_SIZE), tf.reshape(tf.cast(example["target"], tf.float32), [1])


def read_unlabeled_tfrecord(example, return_image_id):
    tfrec_format = {
        "wave": tf.io.FixedLenFeature([], tf.string),
        "wave_id": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrec_format)
    return prepare_image(example["wave"], IMAGE_SIZE), example["wave_id"] if return_image_id else 0


def count_data_items(fileids):
    return len(fileids) * 28000


def count_data_items_test(fileids):
    return len(fileids) * 22600


def prepare_image(wave, dim=256):    
    wave = tf.reshape(tf.io.decode_raw(wave, tf.float64), (3, 4096))    
    normalized_waves = []
    for i in range(3):
        normalized_wave = wave[i] / tf.math.reduce_max(wave[i])
        normalized_waves.append(normalized_wave)
    wave = tf.stack(normalized_waves)
    wave = tf.cast(wave, tf.float32)
    image = create_cqt_image(wave, HOP_LENGTH)
    image = tf.image.resize(image, size=(dim, dim))    
    return tf.reshape(image, (dim, dim, 3))


def get_dataset(files, batch_size=16, repeat=False, shuffle=False, aug=True, labeled=True, return_image_ids=True):    
    ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO, compression_type="GZIP")
    ds = ds.cache() # dataset을 캐시 함으로서, 로컬에 저장하여 효율성 높임. 각 에폭에서만 적용?

    if repeat:
        ds = ds.repeat()

    if shuffle:
        ds = ds.shuffle(1024 * 2)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)

    
    # https://www.tensorflow.org/api_docs/python/tf/data/TFRecordDataset#map
    if labeled == "sampling":
        ds = ds.map(read_id_label_tfrecord, num_parallel_calls=AUTO)
    elif labeled:
        ds = ds.map(read_labeled_tfrecord, num_parallel_calls=AUTO)        
    else:
        ds = ds.map(lambda example: read_unlabeled_tfrecord(example, return_image_ids), num_parallel_calls=AUTO)

    ds = ds.batch(batch_size * REPLICAS)
    
    if aug:
        ds = ds.map(lambda x, y: aug_f(x, y, batch_size * REPLICAS), num_parallel_calls=AUTO)    
    
    ds = ds.prefetch(AUTO)
    
    print(ds)
    return ds  

import os
import math
import random
import re
import warnings
from pathlib import Path
from typing import Optional, Tuple

import efficientnet.tfkeras as efn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_addons as tfa
from scipy.signal import get_window
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

def build_model(size=256, efficientnet_size=0, weights="imagenet", count=0):
    inputs = tf.keras.layers.Input(shape=(size, size, 3))
    
    efn_string= f"EfficientNetB{efficientnet_size}"
    efn_layer = getattr(efn, efn_string)(input_shape=(size, size, 3), weights=weights, include_top=False) # getattr(efn, efn_string) == efn.efn_string 같지만 활용도가 좋음

    x = efn_layer(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    model = tf.keras.Model(inputs=inputs, outputs=x)

    lr_decayed_fn = tf.keras.experimental.CosineDecay(1e-3, count) # learning rate schedule
    opt = tfa.optimizers.AdamW(lr_decayed_fn, learning_rate=1e-4)
    loss = tf.keras.losses.BinaryCrossentropy()
    model.compile(optimizer=opt, loss=loss, metrics=["AUC"])
    model.summary()
    return model
    
        
def get_lr_callback(batch_size=8, replicas=8):
    lr_start   = 1e-4
    lr_max     = 0.000015 * replicas * batch_size
    lr_min     = 1e-7
    lr_ramp_ep = 3
    lr_sus_ep  = 0
    lr_decay   = 0.7
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
            
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
            
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
            
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)
    return lr_callback   

def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

def auto_select_accelerator(): # TPU Setting
    TPU_DETECTED = False
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
        TPU_DETECTED = True
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")

    return strategy, TPU_DETECTED


set_seed(1213)

strategy, tpu_detected = auto_select_accelerator()
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

all_files = get_datapath()

In [ ]:
%%time
files_train_all = np.array(all_files[:5])
print(files_train_all)
kf = KFold(n_splits=5, shuffle=True, random_state=1213)

fp = []
fn = []
tp = []
tn = []
    
tptest = []
tntest = []
    
cnt = 0
    
tpn = 0
tnn = 0
fpn = 0
fnn = 0

for fold, (trn_idx, val_idx) in enumerate(kf.split(files_train_all)):
    files_train = files_train_all[trn_idx]     
    files_valid = files_train_all[val_idx]

    train_image_count = count_data_items(files_train) 
    valid_image_count = count_data_items(files_valid) 
    
    print(fold, trn_idx, val_idx, train_image_count, valid_image_count)    
    tf.keras.backend.clear_session() 
    strategy, tpu_detected = auto_select_accelerator()
    with strategy.scope():
        model = build_model(
                size=IMAGE_SIZE, 
                efficientnet_size=EFFICIENTNET_SIZE,
                weights=WEIGHTS, 
                count=train_image_count // BATCH_SIZE // REPLICAS // 4)
        

    model.load_weights("../input/b3-10epoch-fold0-alldata/alltrain_b3_10epoch_fold0.h5")
    ds_trEval = get_dataset(files_train, labeled="sampling", return_image_ids=False, repeat=False, shuffle=False, batch_size=BATCH_SIZE, aug=False)        
    
    endure_threshold = 0.2
    fnignore = 0
    fpignore = 0
    for wave, wave_id, target in ds_trEval:
        pred = model.predict(wave, verbose=1)
        #print(len(pred), len(wave_id), len(target))        
        cal = (target-pred).numpy()        
        for idx, value in enumerate(cal):        
            if 0.5 < value <= 1: # (1-endure_threshold) fn, if target = 1, predition < 0.1 then ignore
                if value < 1 - endure_threshold:
                    fnn += 1
                    fn.append(wave_id[idx].numpy())
                else:
                    fnignore += 1
            elif 0 < value <= 0.5: # tp
                tpn += 1
                tp.append(wave_id[idx].numpy())
            elif 0 > value > -0.5: # tn
                tnn += 1
                tn.append(wave_id[idx].numpy())    
            elif -1 <= value <= -0.5: # fp, if target = 0, prediction > 0.9 then ignore
                if value > -1 + endure_threshold:
                    fpn +=1
                    fp.append(wave_id[idx].numpy())                
                else:
                    fpignore += 1
            elif value == 0:
                if target == 0: #tn
                    tnn += 1
                    tn.append(wave_id[idx].numpy())
                elif target == 1: #tp
                    tpn += 1
                    tp.append(wave_id[idx].numpy())
                tpn += 1                    
            else:
                print("Error", value, target, idx)
                import sys
                sys.exit(0)
                
    print("tp, tn, fp, fn")
    print(len(tp),len(tn),len(fp),len(fn))
    print(tpn, tnn, fpn, fnn)
    print(tpn + tnn + fpn + fnn)
    print(fpignore, fnignore)
    break     

In [ ]:
%%time
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.python as tfp
from tqdm import tqdm

def get_train_file_path(image_id):
    return "../input/g2net-gravitational-wave-detection/train/{}/{}/{}/{}.npy".format(
        image_id[0], image_id[1], image_id[2], image_id)

def _bytes_feature(value):
    if isinstance(value, tfp.framework.ops.EagerTensor):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def create_tf_example(wave_id: str, wave: bytes, target: int) -> tf.train.Example:
    feature = {
        "wave_id": _bytes_feature(wave_id),
        "wave": _bytes_feature(wave),
        "target": _int64_feature(target)
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))


def write_tfrecord(samlist_pos, samlist_neg, filename: str):    
    options = tf.io.TFRecordOptions("GZIP")
    with tf.io.TFRecordWriter(filename, options=options) as writer:
        print(len(samlist_pos), len(samlist_neg))
        sample_list = dict()
        for i in samlist_pos:
            sample_list[i] = 1
        for i in samlist_neg:
            sample_list[i] = 0        
        sl = list(sample_list.keys())
        random.shuffle(sl)
        
        for i in tqdm(sl):
            wave_id = i # for byte
            wave_dir = get_train_file_path(str(wave_id).split('\'')[1])            
            wave = np.load(wave_dir).tobytes()
            target = sample_list[i]
            tf_example = create_tf_example(wave_id, wave, target)            
            writer.write(tf_example.SerializeToString())
                
train_samples_per_file = 28000
#print(len(fp+fn+tp+tn))
train_number_of_files = len(fp+fn+tp+tn) // train_samples_per_file
#print(train_number_of_files)

tprate = round(len(tp) / len(tp+tn+fp+fn) * 100) 
tnrate = round(len(tn) / len(tp+tn+fp+fn) * 100) 
fprate = round(len(fp) / len(tp+tn+fp+fn) * 100) 
fnrate = round(len(fn) / len(tp+tn+fp+fn) * 100) 
print(tprate, tnrate, fprate, fnrate)
tp_prev = 0
tn_prev = 0
fp_prev = 0
fn_prev = 0

print(train_number_of_files)
#print(len(tp+tn+fp+fn))
for i in range(train_number_of_files):
    tp_cur = tp_prev + int(tprate * train_samples_per_file * 0.01)
    tn_cur = tn_prev + int(tnrate * train_samples_per_file * 0.01)
    fp_cur = fp_prev + int(fprate * train_samples_per_file * 0.01)
    fn_cur = fn_prev + int(fnrate * train_samples_per_file * 0.01)
    
    print(tp_cur, tn_cur, fp_cur, fn_cur)
    filename = f"sampling_train{i}.tfrecords"                                                                           
    a = write_tfrecord(fn[fn_prev:fn_cur] + tp[tp_prev:tp_cur], fp[fp_prev:fp_cur] + tn[tn_prev:tn_cur], filename)    
    tp_prev = tp_cur
    tn_prev = tn_cur
    fp_prev = fp_cur
    fn_prev = fn_cur

<h1> Enjoy Kaggle ! Thanks </h1>